<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>Exploring SAR Time Series Data for Flood Monitoring</b></font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, <a href="http://earthbigdata.com/" target="_blank">Earth Big Data, LLC</a> </b> <br>
<img style="padding:7px;" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /></font>

<font size="3">This notebook introduces you to the time series signatures associated with flooding. The data analysis is doen in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. <br>

<b>This notebook covers the following data analysis concepts:</b>

- How to load time series stacks into Jupyter Notebooks and how to explore image content using basic functions such as mean value calculation and histogram analysis.
- How to extract time series information for individual pixels of an image.
- Typical time series signatures over forests and deforestation sites.
</font>


</font>

<hr>
<font face="Calibri" size="5" color='rgba(200,0,0,0.2)'> <b>Important Notes about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>
<br><br>
</font>


In [ ]:
from IPython.display import Markdown
from IPython.display import display

env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use Create_OSL_Conda_Environments.ipynb to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3">In this notebook we will use the following scientific libraries:

<ol type="1">
    <li> <b><a href="https://pandas.pydata.org/" target="_blank">Pandas</a></b> is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. </li>
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
</font>

In [ ]:
# Check Python version:
import sys
pn = sys.version_info[0]

import os # for chdir, getcwd, path.basename, path.exists
from glob import glob
from math import ceil

import pandas as pd # for DatetimeIndex
from osgeo import gdal # for GetRasterBand, Open, ReadAsArray
import numpy as np #for log10, mean, percentile, power
import pyproj

%matplotlib notebook
import matplotlib.pyplot as plt # for add_subplot, axis, figure, imshow, legend, plot, set_axis_off, set_data,
                                # set_title, set_xlabel, set_ylabel, set_ylim, subplots, title, twinx
import matplotlib.patches as patches  # for Rectangle
import matplotlib.animation as an # for FuncAnimation
from matplotlib import rc

from IPython.display import HTML
plt.rcParams.update({'font.size': 12})

if pn == 2:
    import cStringIO #needed for the image checkboxes
elif pn == 3:
    import io
    import base64
    
import asf_notebook as asfn

# For exporting:
from PIL import Image

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Data Stack</b> </font> <img src="https://www.elcomercio.com/files/article_main/uploads/2017/03/29/58dc6b8cd9665.jpeg" width="350" style="padding:5px;" align="right" /> 

<font size="3"> This notebook will be using a Sentinel-1 data stack (VV only) north of Guayaquil. 

Ecuador and other countries in western South America experienced widespread <b>flooding</b> during the 2016-2017 winter (see picture by Bolívar Velasco/EL COMERCIO). <a href='https://www.eluniverso.com/noticias/2017/03/01/nota/6068059/arboles-se-caen-medio-torrencial-lluvia-ayer'>Guayaquil in Guayas</a> was among the affected regions, as precipitation in March 2017 was well above average. The increased precipitation was associated with a Coastal Niño event.
<br><br>
We will study the extent and progression of the flooding in the year 2016-2017 just north of Guayaquil.
<br><br>
<b>Begin by writing a function to retrieve the absolute paths to each of our tiffs:</b>
</font></font>




In [ ]:
def get_tiff_paths(paths):
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths

<font face="Calibri" size="3"><b>Enter the path to the directory holding your tiffs:</b> </font>

In [ ]:
while True:
    print("Enter the absolute path to the directory holding your tiffs.")
    print("NOTE: the tiffs should all be in the same polarization.")
    tiff_dir = input()
    paths = f"{tiff_dir}/*.tif"
    if os.path.exists(tiff_dir):
        tiff_paths = get_tiff_paths(paths)
        if len(tiff_paths) < 1:
            print(f"{tiff_dir} exists but contains no tifs.")
            print("You will not be able to proceed until tifs are prepared.")
        break
    else:
        print(f"\n{tiff_dir} does not exist.")
        continue

<font face="Calibri" size="3"><b>Determine the path to the analysis directory containing the tiff directory:</b> </font>

In [ ]:
analysis_dir = os.path.dirname(tiff_dir)
print(analysis_dir)

<font face="Calibri" size="3"><b>Create a wildcard path to the tiffs (in VV polarity):</b> </font>

In [ ]:
path_options = [f"{tiff_dir}/*VV.tif*", f"{tiff_dir}/*vv.tif*",
               f"{tiff_dir}/*VH.tif*", f"{tiff_dir}/*vh.tif*"]
for p in path_options:
    pths = glob(p)
    if pths:
        wildcard_path = p
        break
print(wildcard_path)

<font face="Calibri" size="3"><b>Write a function to extract the tiff dates from a wildcard path.</b> Note that product naming conventions are not always consistent so it may be necessary to alter "date = p.split('/')[-1].split("_")[3].split('T')[0]" to parse the date from the filenames correctly.</font>

In [ ]:
def get_dates(pths):
    dates = []
    for p in pths:
        for name_chunk in p.split('/')[-1].split('_'):
            nums = list(range(48, 58))
            if len(name_chunk) == 15 and ord(name_chunk[0]) in nums: 
                date = name_chunk.split('T')[0]
                dates.append(date)
                break
    dates.sort()
    return dates

<font face="Calibri" size="3"><b>Call get_dates() to collect the product acquisition dates:</b></font>

In [ ]:
dates = [ os.path.basename(d).split('_')[0] for d in pths ]
print(dates)

<br>
<font face="Calibri" size="5"> <b> 2. Create the VRT </b> </font> 
<br><br>
<font face="Calibri" size="3"><b>Parse the polarization from a tiff name and define a path to the vrt:</b></font>

In [ ]:
polarization = get_tiff_paths(wildcard_path)[0].split('.')[-2][-2:]
print(polarization)
raster_path = f"{analysis_dir}/raster_stack_{polarization}.vrt"

<font face="Calibri" size="3"><b>Create the virtual raster table for the GeoTiffs:</b></font>

In [ ]:
!gdalbuildvrt -separate $raster_path $wildcard_path

<font face="Calibri" size="3"><b>Create Pandas time index:</b> and print the dates</font>

In [ ]:
time_index = pd.DatetimeIndex(dates)

for jacqdate, acqdate in enumerate(time_index):
    print('{:4d} {}'.format(jacqdate, acqdate.date()),end=' ')
    if (jacqdate % 5 == 4): print()

<br>
<hr>
<font face="Calibri" size="5"> <b> 3. Data exploration with an animation </b> </font> 


<font face="Calibri" size="3"><b>Read the data</b></font>

In [ ]:
img = gdal.Open(raster_path)
band = img.GetRasterBand(1)
raster0 = band.ReadAsArray()
band_number = 0 # Needed for updates
rasterstack = img.ReadAsArray()

<font face="Calibri" size="3">Before analyzing the data, decide whether to use <b>linear or logarithmic scaling</b></font>

In [ ]:
use_dB = False

def convert(raster, use_dB=use_dB):
    # some Python trickery: 
    # if you call the convert function later, you can set the keyword 
    # argument use_dB to True or False
    # if you do not provide a keyword argument, the value that you set
    # above (when defining the function) is used
    if use_dB:
        return 10 * np.log10(raster)
    else:
        return raster


<font face="Calibri" size="3"> Let's create an <b>animation</b> to get an idea of where and when flooding might have occurred.</font>

In [ ]:
%%capture 
figani = plt.figure(figsize=(10, 5))
axani = figani.subplots()
axani.axis('off')

rasterstack_ = convert(rasterstack)

imani = axani.imshow(rasterstack_[0,...], cmap='gray', vmin=np.nanpercentile(rasterstack_, 1), 
               vmax=np.nanpercentile(rasterstack_, 99))
axani.set_title("{}".format(time_index[0].date()))

def animate(i):
    axani.set_title("{}".format(time_index[i].date()))
    imani.set_data(rasterstack_[i,...])

# Interval is given in milliseconds
ani = an.FuncAnimation(figani, animate, frames=rasterstack_.shape[0], interval=300)
rc('animation', embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation

<font face="Calibri" size="3"><b>Render</b></font>

In [ ]:
HTML(ani.to_jshtml())

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u>:  </font> Backscatter dynamics</b> </font>

<font face="Calibri" size="3"> What is the most striking change in February 2017? Can you explain why the backscatter changes the way it does during that period?
</font>
</div>


<hr>
<br>
<font face="Calibri" size="5"> <b> 4. Create Minimum Image to Identify Inundated Areas </b> </font>
<br><br>
<font face="Calibri" size="3"> As flooding is often associated with very low backscater, we first compute the minimum backscatter for each pixel to get a first impression of areas that could have been flooded during the entire period. </font>


<font face="Calibri" size="3"> The following line <b>calculates the minimum backscatter per pixel</b> across the time series: </font>

In [ ]:
temporal_min = np.nanmin(convert(rasterstack), axis=0)

<br>
<font face="Calibri" size="4"> <b> 4.2 Visualize the Minimum Image with Curser Information Included </b> </font>

<font face="Calibri" size="3"> We will now visualize the minimum image in a way that we can move our mouse over the image and visualize the line/sample image coordinates. This will help us create time-series information for the most interesting image locations. 
    
To do so, we <b>first create some helper functions:</b>
</font> 

In [ ]:
class pixelPicker:
    def __init__(self, image, width, height):
        self.x = None
        self.y = None
        self.fig = plt.figure(figsize=(width, height))
        self.ax = self.fig.add_subplot(111, visible=False)
        self.rect = patches.Rectangle(
            (0.0, 0.0), width, height, 
            fill=False, clip_on=False, visible=False
        )
       
        self.rect_patch = self.ax.add_patch(self.rect)
        self.cid = self.rect_patch.figure.canvas.mpl_connect('button_press_event', 
                                                             self)
        self.image = image
        self.plot = self.gray_plot(self.image, fig=self.fig, return_ax=True)
        self.plot.set_title('Select a Point of Interest')
        
        
    def gray_plot(self, image, vmin=None, vmax=None, fig=None, return_ax=False):
        '''
        Plots an image in grayscale.
        Parameters:
        - image: 2D array of raster values
        - vmin: Minimum value for colormap
        - vmax: Maximum value for colormap
        - return_ax: Option to return plot axis
        '''
        if vmin is None:
            vmin = np.nanpercentile(self.image, 1)
        if vmax is None:
            vmax = np.nanpercentile(self.image, 99)
        if fig is None:
           my_fig = plt.figure() 
        ax = fig.add_axes([0.1,0.1,0.8,0.8])
        ax.imshow(image, cmap=plt.cm.gist_gray, vmin=vmin, vmax=vmax)
        if return_ax:
            return(ax)
        
    
    def __call__(self, event):
        print('click', event)
        self.x = event.xdata
        self.y = event.ydata
        for pnt in self.plot.get_lines():
            pnt.remove()
        plt.plot(self.x, self.y, 'ro')


<font face="Calibri" size="3"> Now we are ready to plot the minimum image. <b>Click a point interest for which you want to analyze radar brightness over time</b>: </font>

In [ ]:
fig_xsize = 7.5
fig_ysize = 7.5
my_plot = pixelPicker(temporal_min, fig_xsize, fig_ysize)

<font face="Calibri" size="3"><b>Save the selected coordinates</b>: </font>

In [ ]:
sarloc = (ceil(my_plot.x), ceil(my_plot.y))
print(sarloc)

<br>
<font face="Calibri" size="5"> <b> 5. Plot SAR Brightness Time Series at Point Locations </b> </font>

<font face="Calibri" size="4"> <b> 5.1 SAR Brightness Time Series at Point Locations </b> </font>

<font face="Calibri" size="3"> We will pick a pixel location identified in the SAR image above and plot the time series for this identified point. By focusing on image locations undergoing deforestation, we should see the changes in the radar cross section related to the deforestation event.
    
First, for processing of the imagery in this notebook we generate a list of image handles and retrieve projection and georeferencing information. We also define a function for mapping image pixels to a geographic projection</font> 

In [ ]:
geotrans = img.GetGeoTransform()
proj = img.GetProjection()
xsize = img.RasterXSize
ysize = img.RasterYSize
bands = img.RasterCount
projlatlon = pyproj.Proj('EPSG:4326') # WGS84
projstring = proj.split('[')[-1][:-2].split(',')[-1][1:-1]
projimg = pyproj.Proj(f'EPSG:{projstring}')

def geolocation(x, y=None, latlon=True):
    if len(x) == 2:
        y = x[1]
        x = x[0]
    ref_x=geotrans[0]+sarloc[0]*geotrans[1]
    ref_y=geotrans[3]+sarloc[1]*geotrans[5]
    if latlon:
        ref_y, ref_x = pyproj.transform(projimg, projlatlon, ref_x, ref_y)
    return (ref_x, ref_y)

<font face="Calibri" size="3"> Now, let's <b>pick a rectangle around a center pixel which we selected and defined in variable <i>sarloc</i></b>...</font>

In [ ]:
extent = (5, 5) # choose a 5 by 5 rectangle
latlon = True # if False: return utm coordinates

refsarloc = geolocation(sarloc, latlon=latlon)
projsymbol = '°' if latlon else 'm'

<font face="Calibri" size="3">... and <b>extract the time series</b> for this small area around the selected center pixel in a memory-efficient way (needed for larger stacks):</font> 

In [ ]:
plt.rcParams.update({'font.size': 9})
bs_aggregated = []
for band in range(bands):
    rs = img.GetRasterBand(band+1).ReadAsArray(sarloc[0], sarloc[1], 
                                               extent[0], extent[1])
    rs_mean = convert(np.nanmean(rs))
    bs_aggregated.append(rs_mean)

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
labeldB = 'dB' if use_dB else 'linear'
ax.plot(time_index, bs_aggregated, color='k', marker='o', markersize=3)
ax.set_xlabel('Date')
ax.set_ylabel(f'Sentinel-1 $\gamma^0$ [{labeldB}]')

plt.grid()
_ = fig.suptitle(f'Location: {refsarloc[0]:.3f}{projsymbol} '
                 f'{refsarloc[1]:.3f}{projsymbol}')

# fig.tight_layout() 
figname = (f'RCSTimeSeries-{refsarloc[0]:.3f}{projsymbol} '
           f'{refsarloc[1]:.3f}{projsymbol}.png')
plt.savefig(figname, dpi=300, transparent='true')

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u>:  </font> Explore Time Series at Different Point Locations </b> </font>

<font face="Calibri" size="3"> Can you interpret and attribute the changes at various locations? Apart from the flooding, what other patterns do you observe?
</font>
</div>
<br>
<hr>

<font face="Calibri" size="2"> <i>ExploreSARTimeSeriesFlood_From_Prepared_Data_Stack.ipynb - Version 1.2.0 - April 2021 </i>
    <br>
        <b>Version Changes:</b>
    <ul>
        <li>from osgeo import gdal</li>
    </ul>
    </i>
</font>